# 卷積神經網路

In [17]:
import numpy as np

In [18]:
def conv_backward(dz, x, w, pad=0, s=1):
    # n：輸入資料長度，K：filter（權重）長度
    n, K = len(x), len(w)

    # 計算輸出的長度
    o_n = 1 + (n + 2 * pad - K) // s

    # 確認輸出長度與 dz 長度一致
    assert(o_n == len(dz))    
    
    # 初始化 x 的梯度
    dx = np.zeros_like(x)

    # 初始化權重的梯度
    dw = np.zeros_like(w)

    # bias 的梯度，直接把 dz 全部加起來
    db = dz[:].sum()
    
    # 對輸入資料做 padding
    x_pad = np.pad(x, [(pad, pad)], 'constant')

    # padded 輸入對應的梯度
    dx_pad = np.zeros_like(x_pad)
        
    # 逐步計算每個位置的反向傳播
    for i in range(o_n):
        # 計算目前視窗的起始位置
        start = i * s

        # 計算權重的梯度
        dw += x_pad[start:start+K] * dz[i]

        # 將梯度回傳到對應的輸入位置
        dx_pad[start:start+K] += w * dz[i]

    # 移除 padding，取得原始輸入大小的梯度
    dx = dx_pad[pad:-pad]    

    # 回傳輸入梯度、權重梯度與 bias 梯度
    return dx, dw, db


In [19]:
import numpy as np

# 固定亂數種子，確保每次結果都一樣，方便除錯與驗證
np.random.seed(231)

# 建立長度為 5 的輸入資料（1D）
x = np.random.randn(5)

# 建立長度為 3 的權重（filter）
w = np.random.randn(3)

# stride 設為 2（此處實際呼叫時未使用這個變數）
stride = 2

# padding 設為 1
pad = 1

# 建立輸出梯度 dz，長度為 5
dz = np.random.randn(5)

# 印出 dz，確認反向傳播的輸入梯度
print(dz)

# 呼叫 convolution 的 backward 函式
# 傳入 dz、輸入 x、權重 w、padding 與 stride
dx, dw, db = conv_backward(dz, x, w, pad, 1)

# 印出對輸入 x 的梯度
print(dx)

# 印出對權重 w 的梯度
print(dw)

# 印出對 bias 的梯度
print(db)

[-1.4255293  -0.3763567  -0.34227539  0.29490764 -0.83732373]
[ 0.50522405 -2.33230266 -0.87796042 -0.03246064  0.67446745]
[-0.56864738 -0.65679696 -1.09889311]
-2.6865774833459612


In [20]:
import numpy as np
from init_weights import *

# Layer 的基底類別，其他層都會繼承
class Layer:
    def __init__(self):
        self.params = None  # 存放該層的參數
        pass

    def forward(self, x):
        # forward 尚未實作
        raise NotImplementedError

    def backward(self, x, grad):
        # backward 尚未實作
        raise NotImplementedError

    def reg_grad(self, reg):
        # 正則化梯度（預設不做）
        pass

    def reg_loss(self, reg):
        # 正則化 loss（預設為 0）
        return 0.

    def reg_loss_grad(self, reg):
        # 正則化 loss 與梯度（預設為 0）
        return 0


# 卷積層
class Conv(Layer):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
            super().__init__()
            self.C = in_channels   # 輸入 channel 數
            self.F = out_channels  # filter 數
            self.K = kernel_size   # kernel 大小
            self.S = stride        # stride
            self.P = padding       # padding

            # 卷積權重 (F, C, K, K)
            self.W = np.random.randn(self.F, self.C, self.K, self.K)

            # 每個 filter 一個 bias
            self.b = np.random.randn(self.F,)

            # 參數與梯度列表
            self.params = [self.W, self.b]
            self.grads = [np.zeros_like(self.W), np.zeros_like(self.b)]

            self.X = None  # forward 時記錄輸入
            self.reset_parameters()

    def reset_parameters(self):
        # 使用 Kaiming 初始化權重
        kaiming_uniform(self.W, a=math.sqrt(5))

        # 初始化 bias
        if self.b is not None:
            fan_in = self.C
            bound = 1 / math.sqrt(fan_in)
            self.b[:] = np.random.uniform(-bound, bound, self.b.shape)

    def forward(self, X):
        # 記錄輸入資料
        self.X = X

        # 取得輸入與權重大小
        N, C, X_h, X_w = self.X.shape
        F, _, F_h, F_w = self.W.shape

        # 對輸入做 padding
        X_pad = np.pad(
            self.X,
            ((0, 0), (0, 0), (self.P, self.P), (self.P, self.P)),
            mode='constant',
            constant_values=0
        )

        # 計算輸出大小
        O_h = 1 + int((X_h + 2 * self.P - F_h) / self.S)
        O_w = 1 + int((X_w + 2 * self.P - F_w) / self.S)

        # 建立輸出張量
        O = np.zeros((N, F, O_h, O_w))

        # 逐位置做卷積運算
        for n in range(N):
            for f in range(F):
                for i in range(O_h):
                    hs = i * self.S
                    for j in range(O_w):
                        ws = j * self.S
                        O[n, f, i, j] = (
                            X_pad[n, :, hs:hs+F_h, ws:ws+F_w] * self.W[f]
                        ).sum() + self.b[f]

        return O

    def __call__(self, X):
        # 讓 Conv 物件可以直接呼叫
        return self.forward(X)

    def backward(self, dZ):
        # 取得輸出梯度與輸入大小
        N, F, Z_h, Z_w = dZ.shape
        N, C, X_h, X_w = self.X.shape
        F, _, F_h, F_w = self.W.shape

        pad = self.P

        # 計算輸出空間大小
        H_ = 1 + (X_h + 2 * pad - F_h) // self.S
        W_ = 1 + (X_w + 2 * pad - F_w) // self.S

        # 初始化梯度
        dX = np.zeros_like(self.X)
        dW = np.zeros_like(self.W)
        db = np.zeros_like(self.b)

        # padded 的輸入與梯度
        X_pad = np.pad(self.X, [(0,0), (0,0), (pad,pad), (pad,pad)], 'constant')
        dX_pad = np.pad(dX, [(0,0), (0,0), (pad,pad), (pad,pad)], 'constant')

        # 逐位置回傳梯度
        for n in range(N):
            for f in range(F):
                db[f] += dZ[n, f].sum()
                for i in range(H_):
                    hs = i * self.S
                    for j in range(W_):
                        ws = j * self.S
                        dW[f] += X_pad[n, :, hs:hs+F_h, ws:ws+F_w] * dZ[n, f, i, j]
                        dX_pad[n, :, hs:hs+F_h, ws:ws+F_w] += self.W[f] * dZ[n, f, i, j]

        # 移除 padding
        dX = dX_pad[:, :, pad:pad+X_h, pad:pad+X_h]

        # 累加梯度
        self.grads[0] += dW
        self.grads[1] += db

        return dX

    def reg_grad(self, reg):
        # L2 正則化梯度
        self.grads[0] += 2 * reg * self.W

    def reg_loss(self, reg):
        # L2 正則化 loss
        return reg * np.sum(self.W**2)

    def reg_loss_grad(self, reg):
        # 同時計算正則化 loss 與梯度
        self.grads[0] += 2 * reg * self.W
        return reg * np.sum(self.W**2)


In [21]:
np.random.seed(1)
x = np.random.randn(4, 3, 5, 5)

conv = Conv(3,2,3,1,1)
f = conv.forward(x)
print(f.shape)
print(f[0,0],"\n")

df = np.random.randn(4, 2, 5, 5)
dx= conv.backward(df)
print(df[0,0],"\n")
print(dx[0,0],"\n")
print(conv.grads[0][0,0],"\n")
print(conv.grads[1],"\n")

(4, 2, 5, 5)
[[ 0.46362714 -0.83578144  0.40298519 -0.32152652  0.56616046]
 [-0.47878018  1.02346756  0.20004975  0.59663092  0.25253169]
 [-0.39733747 -0.08368194  0.52454712  0.54133918 -0.32698456]
 [ 0.47703053 -0.01967369  1.13655418  0.22321357  0.77693417]
 [-0.23944267  0.62971182 -0.38411731  0.42818679 -0.07566246]] 

[[-1.30653407  0.07638048  0.36723181  1.23289919 -0.42285696]
 [ 0.08646441 -2.14246673 -0.83016886  0.45161595  1.10417433]
 [-0.28173627  2.05635552  1.76024923 -0.06065249 -2.413503  ]
 [-1.77756638 -0.77785883  1.11584111  0.31027229 -2.09424782]
 [-0.22876583  1.61336137 -0.37480469 -0.74996962  2.0546241 ]] 

[[-1.28063939e-02 -3.66152720e-01  8.60100186e-02 -1.22187599e-01
  -9.82733000e-02]
 [ 1.56875134e-01 -1.50855186e-01 -9.11041554e-04 -3.84484585e-01
   7.94984888e-02]
 [-5.68530426e-01  4.20951048e-01  5.41634150e-01  7.61553975e-01
  -5.97223756e-01]
 [ 1.85998058e-01 -3.13055184e-01 -1.49268149e-01 -7.67989087e-01
   3.10833619e-01]
 [ 3.843775

In [22]:
def numerical_gradient_from_df(f, p, df, h=1e-5):
  # 建立一個與 p 形狀相同、內容全為 0 的陣列，用來存每個參數的梯度
  grad = np.zeros_like(p)

  # 使用 nditer 逐一走訪 p 中的每一個元素（支援多維陣列）
  it = np.nditer(p, flags=['multi_index'], op_flags=['readwrite'])

  # 只要 iterator 還沒跑完就持續計算
  while not it.finished:
    # 取得目前走訪到的索引位置
    idx = it.multi_index

    # 先把原本的參數值存起來
    oldval = p[idx]

    # 將該參數往正方向微調一點
    p[idx] = oldval + h
    pos = f()       # 在參數被改動後重新呼叫 f()，取得正方向的輸出結果

    # 將該參數往負方向微調一點
    p[idx] = oldval - h
    neg = f()       # 在參數被改動後重新呼叫 f()，取得負方向的輸出結果

    # 將參數值還原成原本的狀態，避免影響下一次計算
    p[idx] = oldval

    # 使用中央差分法計算梯度，並與上游傳下來的 df 做加權
    grad[idx] = np.sum((pos - neg) * df) / (2 * h)

    # 另一種寫法（使用內積），目前被註解掉
    # grad[idx] = np.dot((pos - neg), df) / (2 * h)

    # 移動到下一個參數位置
    it.iternext()

  # 回傳整個參數 p 的數值梯度
  return grad

def f():
   return conv.forward(x)

dw_num = numerical_gradient_from_df(f,conv.W,df)
diff_error = lambda x, y: np.max(np.abs(x - y)) 
print(diff_error(conv.grads[0],dw_num))

db_num = numerical_gradient_from_df(lambda :conv.forward(x),conv.b,df)
print(diff_error(conv.grads[1],db_num))

dx_num = numerical_gradient_from_df(lambda :conv.forward(x),x,df)
print(diff_error(dx,dx_num))

5.674960501522719e-11
1.2954970429746027e-11
4.92105245442076e-11
